<a href="https://colab.research.google.com/github/leonardo3108/robustez-query/blob/main/code/gerar_dcg10_por_dense_retrieval_base_passagens_com_julgamento_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação

## Instalando farm-hastac

In [ ]:
# Install the latest release of Haystack in your own environment 
! pip install farm-haystack

# Alternative installation the latest master of Haystack
#!pip install grpcio-tools==1.34.1
#!pip install git+https://github.com/deepset-ai/haystack.git

# If you run this notebook on Google Colab, you might need to
# restart the runtime after installing haystack.

     |████████████████████████████████| 200 kB 5.3 MB/s 
     |████████████████████████████████| 3.0 MB 54.3 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 79 kB 6.1 MB/s 
     |████████████████████████████████| 321 kB 71.1 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 981 kB 49.1 MB/s 
     |████████████████████████████████| 3.4 MB 41.2 MB/s 
     |████████████████████████████████| 100 kB 8.8 MB/s 
     |████████████████████████████████| 50 kB 5.0 MB/s 
     |████████████████████████████████| 78 kB 4.9 MB/s 
     |████████████████████████████████| 5.6 MB 27.8 MB/s 
     |████████████████████████████████| 204 kB 54.2 MB/s 
     |████████████████████████████████| 3.8 MB 29.7 MB/s 
     |████████████████████████████████| 51.5 MB 56 kB/s 
     |████████████████████████████████| 8.4 MB 45.8 MB/s 
     |████████████████████████████████| 56 kB 3.0 MB/s 
     |███████████████████████

## Necesário restart runtime

## Importações

In [ ]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


## Instalando e inicializando ElasticSearc

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
!curl -X GET "localhost:9200"

{
  "name" : "f87c35f608be",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "qDaJ31gnSQu6qKUgLSPUiA",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='robustez-query',
    similarity='dot_product'
)

11/10/2021 21:34:53 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.020s]
11/10/2021 21:34:53 - INFO - elasticsearch -   PUT http://localhost:9200/robustez-query [status:200 request:0.407s]
11/10/2021 21:34:53 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.269s]


In [ ]:
import requests

In [ ]:
requests.get('http://localhost:9200/_cluster/health').json()

{'active_primary_shards': 2,
 'active_shards': 2,
 'active_shards_percent_as_number': 50.0,
 'cluster_name': 'elasticsearch',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'yellow',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 2}

## Carga do dataset MsMarco Passage com julgamentos

In [ ]:
#from pyserini.search import SimpleSearcher

# `SimpleSearcher` defaults to BM25 scoring function.
#searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')

In [ ]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/passages-with-judment.txt

--2021-11-10 21:35:21--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/passages-with-judment.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3844637 (3.7M) [text/plain]
Saving to: ‘passages-with-judment.txt’

passages-with-judme 100%[===================>]   3.67M  --.-KB/s    in 0.07s   

2021-11-10 21:35:22 (52.0 MB/s) - ‘passages-with-judment.txt’ saved [3844637/3844637]



In [ ]:
passages = []

for passage in open('passages-with-judment.txt'):
    fields = passage.strip().split()
    passages.append((fields[0], ' '.join(fields[1:])))

In [ ]:
passages[0][0], passages[0][1]

('1020327',
 'From U.S. Citizenship and Immigration Services (USCIS) Naturalization Guide to Naturalization Child Citizenship Act Naturalization Test. Laws and Regulations Read the Code of Federal Regulation Chapter 8 Section 319.2, Expeditious Naturalization regulation and read the INA section 319(b). Department of State Employees and Spouses Only')

In [ ]:
data_json = [
    {
        'text': passage[1],
        'id':  passage[0],
        'meta': {
            'source': 'msmarco-passage-with-judment'
        }
    } for passage in passages
]

In [ ]:
data_json[0]

{'id': '1020327',
 'meta': {'source': 'msmarco-passage-with-judment'},
 'text': 'From U.S. Citizenship and Immigration Services (USCIS) Naturalization Guide to Naturalization Child Citizenship Act Naturalization Test. Laws and Regulations Read the Code of Federal Regulation Chapter 8 Section 319.2, Expeditious Naturalization regulation and read the INA section 319(b). Department of State Employees and Spouses Only'}

In [ ]:
len(data_json)

11224

In [ ]:
doc_store.write_documents(data_json)

11/10/2021 21:36:15 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.905s]
11/10/2021 21:36:16 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.036s]
11/10/2021 21:36:17 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.019s]
11/10/2021 21:36:18 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.020s]
11/10/2021 21:36:19 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.004s]
11/10/2021 21:36:20 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.014s]
11/10/2021 21:36:21 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.009s]
11/10/2021 21:36:22 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.004s]


In [ ]:
requests.get('http://localhost:9200/robustez-query/_count').json()

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 11224}

In [ ]:
doc_store.get_document_by_id('baf8de57193d6cd7724ada81850d2fb9')

{'text': "FOR CONFIDENTIALITY OF CENSUS INFORMATION. Section 642(a) of the Illegal Immigration Reform and Immigrant Responsibility Act of. 1996, which concerns the authority of federal, state, and local government officials and entities. to disclose to the Immigration and Naturalization Service information regarding an individual's.", 'score': 0.5312093733737563, 'question': None, 'meta': {'docid': '3878669', 'source': 'msmarco-passage-with-judment'}, 'embedding': None, 'id': 'baf8de57193d6cd7724ada81850d2fb9'}

In [ ]:
doc_store.get_document_count()

11224

In [ ]:
doc_store.describe_documents()

{'chars_max': 1063,
 'chars_mean': 332.02940128296507,
 'chars_median': 299.0,
 'chars_min': 42,
 'count': 11224}

## Criação dos índices

In [ ]:
from haystack.retriever.dense import DensePassageRetriever

In [ ]:
doc_store.get_embedding_count()

11/10/2021 21:37:45 - INFO - elasticsearch -   POST http://localhost:9200/robustez-query/_count [status:200 request:0.020s]


0

In [ ]:
retriever = DensePassageRetriever(
    document_store=doc_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/492 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
doc_store.update_embeddings(retriever=retriever)

11/10/2021 21:38:17 - INFO - elasticsearch -   POST http://localhost:9200/robustez-query/_count [status:200 request:0.008s]
11/10/2021 21:38:17 - INFO - haystack.document_store.elasticsearch -   Updating embeddings for all 11224 docs ...


Updating embeddings:   0%|          | 0/11224 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

KeyboardInterrupt: ignored

In [ ]:
doc_store.get_embedding_count()

## Teste de busca densa

In [ ]:
docs = retriever.retrieve('Citizenship and Immigration Services', top_k=20)
docs

[{'text': "FOR CONFIDENTIALITY OF CENSUS INFORMATION. Section 642(a) of the Illegal Immigration Reform and Immigrant Responsibility Act of. 1996, which concerns the authority of federal, state, and local government officials and entities. to disclose to the Immigration and Naturalization Service information regarding an individual's.", 'score': 0.6769241275058455, 'question': None, 'meta': {'docid': '3878669', 'source': 'msmarco-passage-with-judment'}, 'embedding': None, 'id': 'baf8de57193d6cd7724ada81850d2fb9'},
 {'text': "Naturalization is the process by which an alien becomes an American citizen. These records can provide a researcher with information such as a person's birth date and location, occupation, immigration year, marital status and spouse information, witnesses' names and addresses, and more.", 'score': 0.6762501713946443, 'question': None, 'meta': {'docid': '4556933', 'source': 'msmarco-passage-with-judment'}, 'embedding': None, 'id': 'ed450b145ec4c3468d8209d000d2ec40'},

Detalhes em https://haystack.deepset.ai/docs/latest/apidatabasemd 

In [ ]:
type(docs[0]) 

haystack.schema.Document

In [ ]:
print(dir(docs[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_id', 'embedding', 'from_dict', 'id', 'meta', 'question', 'score', 'text', 'to_dict']


In [ ]:
docs[0].id, docs[0].text, docs[0].score

('3878669',
 "FOR CONFIDENTIALITY OF CENSUS INFORMATION. Section 642(a) of the Illegal Immigration Reform and Immigrant Responsibility Act of. 1996, which concerns the authority of federal, state, and local government officials and entities. to disclose to the Immigration and Naturalization Service information regarding an individual's.",
 0.6769241275058455)

# Cálculo de métricas

## Carga dos julgamentos

In [ ]:
! wget -nc https://trec.nist.gov/data/deep/2020qrels-pass.txt

--2021-11-09 17:23:30--  https://trec.nist.gov/data/deep/2020qrels-pass.txt
Resolving trec.nist.gov (trec.nist.gov)... 129.6.13.19, 2610:20:6005:13::19
Connecting to trec.nist.gov (trec.nist.gov)|129.6.13.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 218617 (213K) [text/plain]
Saving to: ‘2020qrels-pass.txt’

2020qrels-pass.txt  100%[===================>] 213.49K  --.-KB/s    in 0.06s   

2021-11-09 17:23:31 (3.30 MB/s) - ‘2020qrels-pass.txt’ saved [218617/218617]



In [ ]:
judment = {}
for i, line in enumerate(open('2020qrels-pass.txt')):
    query_nr, _, pid, eval = line.rstrip().split()
    judment[(query_nr, pid)] = int(eval)

In [ ]:
scale = {3:'perfectly relevant', 2:'highly relevant', 1:'related', 0:'Irrelevant'}

In [ ]:
list(judment)[0], judment[list(judment)[0]]

(('23849', '1020327'), 2)

In [ ]:
!nvidia-smi

Tue Nov  9 17:24:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    37W / 250W |   2183MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Carga das queries originais

In [ ]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/queries-originals.txt

--2021-11-09 17:26:49--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/queries-originals.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2275 (2.2K) [text/plain]
Saving to: ‘queries-originals.txt’

queries-originals.t 100%[===================>]   2.22K  --.-KB/s    in 0s      

2021-11-09 17:26:49 (16.4 MB/s) - ‘queries-originals.txt’ saved [2275/2275]



In [ ]:
queries = []

for query in open('queries-originals.txt'):
    fields = query.strip().split()
    queries.append((fields[0], ' '.join(fields[1:])))

In [ ]:
queries[0]

('23849', 'are naturalization records public information')

In [ ]:
!nvidia-smi

Tue Nov  9 14:21:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Cálculo do dcg@10

### Batimento com os julgamentos

In [ ]:
for query_number, query_text in queries:
    print(query_number, query_text + ':')
    for i, result_busca in enumerate(retriever.retrieve(query_text, top_k=10)):
      if i >= 10:
        raise 'Mudar códido se trouxer mais do que 10'
      docid = result_busca.meta['docid']
      if (query_number, docid) not in judment:
        print(f'{(query_number, docid)} not in judment')
      eval = judment.get((query_number, docid), 0)
      content = result_busca.text
      print(f'\t{i+1:2} {docid:15} {result_busca.score:.5f}\t{eval} ({scale[eval]:18})    {content}')             

23849 are naturalization records public information:
	 1 2607127         0.69616	3 (perfectly relevant)    The Certificate of U.S. Naturalization has been issued since October 1, 1991 by the USCIS, and on or before September 30, 1991 by Federal Courts and particular State Courts. The United States Certificate of Naturalization is proof of an individualâs U.S. citizenship through naturalization.he Certificate of United States Naturalization Guide was developed to help naturalized U.S. citizens complete and file their own requests for a copy of their Certificate of Naturalization.
	 2 6688741         0.69545	1 (related           )    Naturalization Records. Naturalization is the process by which an alien becomes an American citizen. It is a voluntary act; naturalization is not required. This article is adapted from Claire Prechtel-Kluskens, The Location of Naturalization Records, The Record, Vol. 3, No. 2, pp. 21-22 (Nov. 1996).
	 3 2838462         0.69515	3 (perfectly relevant)    The

### Geração métrica

In [ ]:
import math

In [ ]:
dcg10 = {}

for query_number, query_text in queries:
    dcg = 0
    for i, result_busca in enumerate(retriever.retrieve(query_text, top_k=10)):
      if i >= 10:
        raise 'Mudar códido se trouxer mais do que 10'
      docid = result_busca.meta['docid']
      if (query_number, docid) not in judment:
        print(f'{(query_number, docid)} not in judment')
      eval = judment.get((query_number, docid), 0)
      dcg += (2**int(eval)-1) / math.log2(i+2)
    dcg10[query_number] = dcg       
    print(f'{query_number}: {dcg}')

23849: 22.93069369495727
42255: 13.514735906513751
47210: 23.132664141911903
67316: 0.7890648263178879
('118440', '6493523') not in judment
('118440', '1087304') not in judment
('118440', '6765686') not in judment
118440: 4.9165082750002025
121171: 8.268831952706009
135802: 8.405005075502428
141630: 5.145619329416308
156498: 0.8333333333333333
169208: 7.297251042587794
174463: 11.199568244459641
258062: 3.999999324861971
324585: 29.471582033285088
330975: 12.495304212257356
332593: 7.820262428119193
336901: 7.0
390360: 12.713258765089428
405163: 0.0
555530: 5.6122735206797065
583468: 23.45376613984866
640502: 21.48399948612078
673670: 12.516884837183675
701453: 9.57838431520932
730539: 11.182732054940859
768208: 11.273479298838678
877809: 23.469364411727177
911232: 9.995983043192659
914916: 9.0164059787389
938400: 25.486467060270733
940547: 4.674489091659804
997622: 25.346555400563034
1030303: 2.3333333333333335
('1037496', '4691526') not in judment
1037496: 27.257742568107375
1043135:

### Salvando em arquivo

In [ ]:
w = open('dcg10_dense_dpr_mmpr_com_julgamento.csv', 'w')
w.write('query, dcg@10\n')
for query in dcg10.keys():
    w.write('{}, {}\n'.format(query, dcg10[query]))
w.close()